# Введение в искусственные нейронные сети
# Урок 5. Сверточные нейронные сети

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

# установка параметров нейросети
batch_size = 32
num_classes = 10
epochs = 1
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# разделение тренировочной и тестовой выборки
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'тренировочные примеры')
print(x_test.shape[0], 'тестовые примеры')

# преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# инициализация RMSprop optimizer
opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation в реальном времени')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=0, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0., 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)
  # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center= True,
        samplewise_center=True,
        featurewise_std_normalization=True,
        samplewise_std_normalization=True,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=0, 
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0., 
        zoom_range=0., 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)

    # запуск data augmentation через fit
    #datagen.fit(x_train)

    # запуск data augmentation через fit_generator
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# сохранение модели и весов
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('сохранить обученную модель как %s ' % model_path)

# проверка работы обученной модели
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

170498071/170498071 [==============================] - 3s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 тренировочные примеры
10000 тестовые примеры


/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Использование data augmentation в реальном времени


<ipython-input-1-e8cffeb38a1a>:127: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(datagen.flow(x_train, y_train,
/usr/local/lib/python3.9/dist-packages/keras/preprocessing/image.py:1861: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/keras/preprocessing/image.py:1871: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


1563/1563 [==============================] - 64s 34ms/step - loss: 1.8759 - accuracy: 0.3065 - val_loss: 2.1173 - val_accuracy: 0.2070
сохранить обученную модель как /content/saved_models/keras_cifar10_trained_model.h5 
313/313 [==============================] - 1s 3ms/step - loss: 2.1173 - accuracy: 0.2070
Test loss: 2.1172571182250977
Test accuracy: 0.2070000022649765


## Практическое задание

<ol>
    <li>Попробовать улучшить точность распознования образов cifar 10 сверточной нейронной сетью, рассмотренной на уроке. Приложить анализ с описанием того, что улучшает работу нейронной сети и что ухудшает.
    </li>
    <li>Описать также в анализе какие необоходимо внести изменения  в получившуюся у вас нейронную сеть если бы ей нужно было работать не с cifar10, а с MNIST, CIFAR100 и IMAGENET.
    </li>
</ol>

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

# Функция, которая создает модель
def create_model():
    model = Sequential()

    # слои нейросети отвественные за свертку и max-pooling
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # полносвязные слои нейронной сети
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    # инициализация RMSprop optimizer
    opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

    # компиляция модели
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

# создаем экземпляр модели для использования с Grid Search
model = KerasClassifier(build_fn=create_model, verbose=0)

# определяем параметры, которые будут оптимизироваться
batch_size = [32, 64, 128]
epochs = [10, 20]
param_grid = dict(batch_size=batch_size, epochs=epochs)

# запускаем Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(x_train, y_train)

# выводим результаты
print("Лучший результат: %f с использованием %s" % (grid_result.best_score_, grid_result.best_params_))
# тоже самое можно провернуть и с другими параметрами (кол-во слоев сети, размер фильтра и тд)

<ipython-input-2-879bccd7823d>:42: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)
/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Лучший результат: 0.721520 с использованием {'batch_size': 32, 'epochs': 20}


## Дополнительные материалы

<ol>
    <li>https://keras.io/layers/convolutional/</li>
    <li>https://keras.io/layers/pooling/</li>
    <li>https://keras.io/preprocessing/image/</li>
</ol>

## Используемая литература 

Для подготовки данного методического пособия были использованы следующие ресурсы:
<ol>
    <li>https://keras.io</li>
    <li>Шакла Н. — Машинное обучение и TensorFlow 2019</li>
    <li>Николенко Сергей Игоревич, Кадурин А. А. - Глубокое обучение. Погружение в мир нейронных сетей  2018</li>
    <li>Francois Chollet - Deep Learning with Python 2018</li>
    <li>Alex Krizhevsky, Ilya Sutskever, Geoffrey E. Hinton - ImageNet Classification with Deep Convolutional Neural Networks</li>
    <li>Karen Simonyan, Andrew Zisserman - Very Deep Convolutional Networks for Large-Scale Image Recognition</li>
    <li>Википедия</li>    
</ol>

In [ ]:
#hw task 1
from itertools import product

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.optimizers import SGD, RMSprop, Adam
from keras.layers import BatchNormalization
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# Функция, которая создает модель
def create_model(activation, filter, optimizer, learn_rate, bn=False):

    # filter = 32
    # optimizer = Adam
    # learn_rate = 0.1
    # bn = False

    model = Sequential()

    # слои нейросети отвественные за свертку и max-pooling
    model.add(Conv2D(filter, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation(activation))
    if bn:
      model.add(BatchNormalization()) 
    model.add(Conv2D(filter, (3, 3)))
    model.add(Activation(activation))
    if bn:
      model.add(BatchNormalization()) 
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(2*filter, (3, 3), padding='same'))
    model.add(Activation(activation))
    if bn:
      model.add(BatchNormalization()) 
    model.add(Conv2D(2*filter, (3, 3)))
    model.add(Activation(activation))
    if bn:
      model.add(BatchNormalization()) 
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # полносвязные слои нейронной сети
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation(activation))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    # инициализация optimizer
    opt = optimizer(learning_rate=learn_rate) # , decay=1e-6

    # компиляция модели
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy']) # , error_score='raise'
    return model

activations = ['relu', 'elu', 'selu']
filters = [32, 64]
optimizers = [SGD, RMSprop, Adam]
learn_rates = [0.001, 0.01, 0.1] # , 0.2, 0.3
bns = [False, True] # BatchNorm

for activision, filter, optimizer, learn_rate, bn in product(activations, filters, optimizers, learn_rates, bns):
    print(f'{activision=}, {filter=}, {optimizer=}, {learn_rate=}, {bn=}')
    # создаем экземпляр модели для использования с Grid Search
    model = KerasClassifier(build_fn=create_model, activation=activision, filter=filter, optimizer=optimizer, learn_rate=learn_rate, verbose=1)
    # model = create_model('relu')

    # определяем параметры, которые будут оптимизироваться
    # momentum = [0.0, 0.2, 0.4] # , 0.6, 0.8, 0.9
    batch_size = [32, 64, 128]
    epochs = [1, 2]
    param_grid = dict(batch_size=batch_size, epochs=epochs) # optimizer=optimizers, , optimizer__learning_rate=learn_rate, optimizer__momentum=momentum

    # запускаем Grid Search
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=None, cv=3) # n_jobs=-1
    grid_result = grid.fit(x_train, y_train)

    # выводим результаты
    print("Лучший результат: %f с использованием %s" % (grid_result.best_score_, grid_result.best_params_))
    # тоже самое можно провернуть и с другими параметрами (кол-во слоев сети, размер фильтра и тд)

activision='relu', filter=32, optimizer=<class 'keras.optimizers.legacy.gradient_descent.SGD'>, learn_rate=0.001, bn=False
521/521 [==============================] - 2s 3ms/step - loss: 2.2959 - accuracy: 0.1537
Epoch 1/2
1042/1042 [==============================] - 6s 5ms/step - loss: 2.3034 - accuracy: 0.1035
Epoch 2/2
521/521 [==============================] - 2s 4ms/step - loss: 2.2957 - accuracy: 0.1793
Epoch 1/2
1042/1042 [==============================] - 5s 5ms/step - loss: 2.3005 - accuracy: 0.1114
Epoch 2/2
521/521 [==============================] - 2s 3ms/step - loss: 2.2678 - accuracy: 0.1865
Epoch 1/2
1042/1042 [==============================] - 6s 5ms/step - loss: 2.3040 - accuracy: 0.1036
Epoch 2/2
261/261 [==============================] - 1s 4ms/step - loss: 2.2958 - accuracy: 0.1350
Epoch 1/2
521/521 [==============================] - 4s 7ms/step - loss: 2.3051 - accuracy: 0.1041
Epoch 2/2
261/261 [==============================] - 1s 4ms/step - loss: 2.2901 - accurac

In [ ]:
# task 1
# подбор опитмальных параметров обучения сети не завершился из-за лимита ресурсов на бесплатном аккунте google colab, но из имеющихся данных уже можно сделать определенные выводы, а именно

# что улучшает работу нейронной сети 
# - использование фанкции активации Relu и ее подобных
# - использование окна размером 3х3
# - увеличение количества фильтров
# - использование оптимизатора Adam
# - поэтапное уменьшение learning rate
# - увеличение batch size при наличии ОП на gpu
# - добавление BatchNormalization
# - увеличение количества эпох 

#и что ухудшает.
# - увеличение количества эпох - отслеживать переобучение
# - высокий learning rate


# task 2
# Нужно модифицировать входной свёрточный слой по размеру изображения, выходной полносвязный слой сети по количевству классов:
# Для CIFAR100 в полносвязном слое должно быть 100 нейронов на 100 классов
# Для MNIST входной сверточный Conv2D(28, (3, 3), по размеру картинки, выход на 10 классов
# Для IMAGENET с размером 469x387, можно переделать в 256x256, вход Conv2D(256, (3, 3), выход на 1000 нейронов, по количеству классов